<font color='red'>
<br>Content:
    
1. [>introduction](#1)    
    * [preliminary information](#2)
    * [dataset](#3)    
1. [Normalization](#4)    
    * [Normalization](#5)
    * [Train/Test Split](#6)
    * [Initialize and Sigmoid Func](#7)
    * [Sigmoid Func](#8)
    * [Updating(Learning)](#9)
    * [Prediction](#10)
    * [Logistic Regression](#11)
    * [Sklearn with Logistic Regresion](#12)


<a id="1"></a> <br>
# >introduction

<a id="2"></a> <br>
# preliminary information

1. * Logistic Regression ( Lojistik Regresyon ) sınıflandırma işlemi yapmaya yarayan bir regresyon yöntemidir
1. * For example,
*     To predict whether an email is spam (1) or (0)
*     Whether the tumor is malignant (1) or not (0)
1. Consider a scenario where we need to classify whether an email is spam or not. If we use linear regression for this problem, there is a need for setting up a threshold based on which classification can be done. Say if the actual class is malignant, predicted continuous value 0.4 and the threshold value is 0.5, the data point will be classified as not malignant which can lead to serious consequence in real time.
1. From this example, it can be inferred that linear regression is not suitable for classification problem. Linear regression is unbounded, and this brings logistic regression into picture. Their value strictly ranges from 0 to 1.


<a ><img src="https://miro.medium.com/max/770/1*UgYbimgPXf6XXxMy2yqRLw.png" alt="1" border="0">


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="3"></a> <br>
# dataset

In [ ]:
data=pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
data.columns

In [ ]:
data.head(10)

In [ ]:
data.info()

<a ><img src="http://preview.ibb.co/cxP63H/5.jpg" alt="1" border="0">
    
    
* Let's try to explain how Logistic Regression works on the chart above.
* We have visuals with 4096 pixels in size and these images also have 0 and 1 values ​​in different drawings.

* Each pixel creates our input values, these go up to px1, px2… px4096.

* We multiply each input value by the weights values.
(px1w1 + px2w2 +… px4096 * w4096)
We add the value we get by bias and get a z value.

* z = (px1w1 + px2 + w2 +… px4096w4096) + b
We subject the obtained z-value to the Sigmoid Function. The Sigmoid Function generates us values ​​between 0 and 1.

* The Sigmoid process is now finished, we have a predict value by the model. We call it y_head.

* If y_head is greater than 0.5, the result is 1, otherwise 0.

* When we ask the model to recognize 0's and 1's, and return us false predictions, we get a loss function.

* When we collect all the loss functions, we get the cost (cost) value.

* We call the processes starting from the first step to the last step as Forward Propagation.

* Ok, now we have created the model, but we had too many errors. Why? Because at first we assigned the first parameter (weights and bias) values ​​and the cost value turned out to be an absurd or high value.

* We should minimize the cost value so that the model works with the best performance. What are we doing for this? Back Propagation.

* With the back propagation algorithm, weights and bias are now determined according to the cost value. Then we do the same again. Until the cost value reaches the minimum value, we understand this when it starts to repeat itself, that is, when it approaches 0.

* Weights: Weight or coefficient of each pixel,
* Bias: Cut-off value.
    
> I hope these explanations were useful.
 
* Let's dump this into code now ;)    

In [ ]:
y=data.Outcome.values

x_data=data.drop("Outcome",axis=1)

<a id="4"></a> <br>
# Normalization

In [ ]:
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data)).values

<a id="5"></a> <br>
# Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2,random_state=42)

x_train = x_train.T
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

<a id="6"></a> <br>
# Initialize and Sigmoid Func

In [ ]:
# dimension = 30
def initialize_weights_and_bias(dimension):
    
    w = np.full((dimension,1),0.01)
    b = 0.0
    return w,b


# w,b = initialize_weights_and_bias(30)


<a id="7"></a> <br>
# Sigmoid Func

In [ ]:
def sigmoid(z):
    
    y_head = 1/(1+ np.exp(-z))
    return y_head
# print(sigmoid(0))

<a id="8"></a> <br>
# Forward Backward Propagation

In [ ]:
def forward_backward_propagation(w,b,x_train,y_train):
    # forward propagation
    z = np.dot(w.T,x_train) + b
    y_head = sigmoid(z)
    loss = -y_train*np.log(y_head)-(1-y_train)*np.log(1-y_head)
    cost = (np.sum(loss))/x_train.shape[1]      # x_train.shape[1]  is for scaling
    
    # backward propagation
    derivative_weight = (np.dot(x_train,((y_head-y_train).T)))/x_train.shape[1] # x_train.shape[1]  is for scaling
    derivative_bias = np.sum(y_head-y_train)/x_train.shape[1]                 # x_train.shape[1]  is for scaling
    gradients = {"derivative_weight": derivative_weight, "derivative_bias": derivative_bias}
    
    return cost,gradients

<a id="9"></a> <br>
# Updating(Learning)

In [ ]:
def update(w, b, x_train, y_train, learning_rate,number_of_iterarion):
    cost_list = []
    cost_list2 = []
    index = []
    
    # updating(learning) parameters is number_of_iterarion times
    for i in range(number_of_iterarion):
        # make forward and backward propagation and find cost and gradients
        cost,gradients = forward_backward_propagation(w,b,x_train,y_train)
        cost_list.append(cost)
        # lets update
        w = w - learning_rate * gradients["derivative_weight"]
        b = b - learning_rate * gradients["derivative_bias"]
        if i % 10 == 0:
            cost_list2.append(cost)
            index.append(i)
            print ("Cost after iteration %i: %f" %(i, cost))
            
    # we update(learn) parameters weights and bias
    parameters = {"weight": w,"bias": b}
    plt.plot(index,cost_list2)
    plt.xticks(index,rotation='vertical')
    plt.xlabel("Number of Iterarion")
    plt.ylabel("Cost")
    plt.show()
    return parameters, gradients, cost_list

<a id="10"></a> <br>
# Prediction

In [ ]:
def predict(w,b,x_test):
    # x_test is a input for forward propagation
    z = sigmoid(np.dot(w.T,x_test)+b)
    Y_prediction = np.zeros((1,x_test.shape[1]))
    # if z is bigger than 0.5, our prediction is sign one (y_head=1),
    # if z is smaller than 0.5, our prediction is sign zero (y_head=0),
    for i in range(z.shape[1]):
        if z[0,i]<= 0.5:
            Y_prediction[0,i] = 0
        else:
            Y_prediction[0,i] = 1

    return Y_prediction

<a id="11"></a> <br>
# Logistic Regression

In [ ]:
def logistic_regression(x_train, y_train, x_test, y_test, learning_rate ,  num_iterations):
    # initialize
    dimension =  x_train.shape[0]  # that is 30
    w,b = initialize_weights_and_bias(dimension)
    # do not change learning rate
    parameters, gradients, cost_list = update(w, b, x_train, y_train, learning_rate,num_iterations)
    
    y_prediction_test = predict(parameters["weight"],parameters["bias"],x_test)

    # Print test Errors
    print("test accuracy: {} %".format(100 - np.mean(np.abs(y_prediction_test - y_test)) * 100))
    
logistic_regression(x_train, y_train, x_test, y_test,learning_rate = 1, num_iterations = 500)  

**** Our 1st method correct estimate rate = 77.27272727272728%.

<a id="12"></a> <br>
# Sklearn with Logistic Regresion 

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train.T,y_train.T)
print("test accuracy {}".format(lr.score(x_test.T,y_test.T)))

**** Our 2st method correct estimate rate = 76.62337662337663%.